<a href="https://colab.research.google.com/github/vuchicago/Python/blob/master/Redwood_MS_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
####MOUNT DRIVE
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [33]:
import os

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
base_dir="/content/gdrive/My Drive/Python/Logistics"
ms_dir=os.path.join(base_dir,"Managed Services")
email_dir=os.path.join(ms_dir,"Outlook Emails")
email_dir_val=os.path.join(ms_dir,"Outlook Emails (Validation)")



In [ ]:
os.chdir(email_dir_val)

!pip install extract-msg
!pip install imapclient
import extract_msg


In [7]:
os.chdir(ms_dir)

In [4]:
####import json file
import json
with open('MS Email Data_output_Apr_01_2021_v2.json') as json_data:
    data = json.load(json_data)##

##data contains entities and text.  Within entities, you have end_idx, start_idx, text, and type.  You want end_idx, start_idx, type


In [14]:
###Code to read json file and store it into dictionary for model to read
entity_data={'entity':[]}
for i in range(1,len(data)):
  partial_data={'entities':[]}
  for entities in data[i]['entities']:
    partial_data['entities'].append((entities['start_idx'],entities['end_idx'],entities['type']))
  entity_data["entity"].append(partial_data)

entity_data['entity']

[{'entities': [(166, 176, 'Pallet Count'),
   (178, 190, 'Weight'),
   (201, 225, 'Delivery Address')]},
 {'entities': [(15, 48, 'Pickup Address'),
   (80, 138, 'Delivery Address'),
   (141, 150, 'PO Number'),
   (153, 162, 'Pallet Count'),
   (174, 183, 'Weight')]},
 {'entities': [(80, 91, 'PO Number'),
   (102, 111, 'Pallet Count'),
   (124, 131, 'Weight'),
   (220, 273, 'Pickup Address'),
   (320, 387, 'Delivery Address')]},
 {'entities': [(51, 103, 'Pickup Address'),
   (127, 138, 'PO Number'),
   (155, 164, 'Pallet Count'),
   (165, 172, 'Weight')]},
 {'entities': [(66, 108, 'Pickup Address'),
   (132, 143, 'PO Number'),
   (174, 183, 'Pallet Count'),
   (184, 192, 'Weight')]},
 {'entities': [(67, 102, 'Pickup Address'),
   (130, 141, 'PO Number'),
   (158, 167, 'Pallet Count'),
   (168, 175, 'Weight')]},
 {'entities': [(51, 103, 'Pickup Address'),
   (123, 134, 'PO Number'),
   (164, 173, 'Pallet Count'),
   (174, 182, 'Weight')]},
 {'entities': [(50, 114, 'Pickup Address'),
   (

In [20]:
a=data[1]['text']
b=data[2]['text']
c=data[3]['text']


In [21]:
##annotate the data
train_data=[
    (a,entity_data['entity'][0]),
    (b,entity_data['entity'][1]),
    (c,entity_data['entity'][2])
]

In [31]:
for i,j in train_data:
  print(j)

{'entities': [(166, 176, 'Pallet Count'), (178, 190, 'Weight'), (201, 225, 'Delivery Address')]}
{'entities': [(80, 91, 'PO Number'), (102, 111, 'Pallet Count'), (124, 131, 'Weight'), (220, 273, 'Pickup Address'), (320, 387, 'Delivery Address')]}
{'entities': [(15, 48, 'Pickup Address'), (80, 138, 'Delivery Address'), (141, 150, 'PO Number'), (153, 162, 'Pallet Count'), (174, 183, 'Weight')]}


In [19]:
######combine the text with the corresponding labeled entities  (Annotate the data)
train_data=[]

for i in range(0,len(entity_data['entity'])):
  train_data.append((data[i+1]['text'],entity_data['entity'][i]))

In [20]:
for i,j in train_data:
  print(i,j)

 {'entities': [(166, 176, 'Pallet Count'), (178, 190, 'Weight'), (201, 225, 'Delivery Address')]}
 {'entities': [(15, 48, 'Pickup Address'), (80, 138, 'Delivery Address'), (141, 150, 'PO Number'), (153, 162, 'Pallet Count'), (174, 183, 'Weight')]}
 {'entities': [(80, 91, 'PO Number'), (102, 111, 'Pallet Count'), (124, 131, 'Weight'), (220, 273, 'Pickup Address'), (320, 387, 'Delivery Address')]}
 {'entities': [(51, 103, 'Pickup Address'), (127, 138, 'PO Number'), (155, 164, 'Pallet Count'), (165, 172, 'Weight')]}
 {'entities': [(66, 108, 'Pickup Address'), (132, 143, 'PO Number'), (174, 183, 'Pallet Count'), (184, 192, 'Weight')]}
 {'entities': [(67, 102, 'Pickup Address'), (130, 141, 'PO Number'), (158, 167, 'Pallet Count'), (168, 175, 'Weight')]}
 {'entities': [(51, 103, 'Pickup Address'), (123, 134, 'PO Number'), (164, 173, 'Pallet Count'), (174, 182, 'Weight')]}
 {'entities': [(50, 114, 'Pickup Address'), (142, 153, 'PO Number'), (171, 180, 'Pallet Count'), (181, 189, 'Weight')]}
 

In [21]:
!pip install spacy

import spacy
# Load Packages
from __future__ import unicode_literals, print_function

import plac #  wrapper over argparse
import random
from pathlib import Path
import spacy
from tqdm import tqdm # loading bar


In [62]:
import en_core_web_sm

In [95]:
# sets the model, output directory and training iterations 
#model = "en_core_web_sm"
model=None
output_dir=ms_dir


# Checks to see if there is a current model or no model. In this case I will be starting with a blank model 

if model is not None:
    ner_model = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
# this will create a blank english model
    ner_model = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")

Created blank 'en' model


In [97]:

# create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in ner_model.pipe_names:
    ner = ner_model.create_pipe('ner')
    ner_model.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = ner_model.get_pipe('ner')

In [ ]:
n_iter=40

# add labels, Trains data based on annotations 
for _,annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])  ###get the labels, which are in the third column of ent (start_num, end_num, labels)

    # get names of other pipes to disable them during training
other_pipes = [pipe for pipe in ner_model.pipe_names if pipe != 'ner']
with ner_model.disable_pipes(*other_pipes):  # only train NER
    optimizer = ner_model.begin_training()
    for itn in range(n_iter):
        random.shuffle(train_data)
        losses = {}
        for text, annotations in tqdm(train_data):
            ner_model.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.2,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

  7%|▋         | 2/27 [00:00<00:01, 13.10it/s]

{'ner': 52.55899437362077}


  7%|▋         | 2/27 [00:00<00:01, 13.31it/s]

{'ner': 53.50498380808208}


  7%|▋         | 2/27 [00:00<00:01, 13.85it/s]

...

  7%|▋         | 2/27 [00:00<00:01, 14.66it/s]

{'ner': 54.96993003001016}


  4%|▎         | 1/27 [00:00<00:02,  9.30it/s]

{'ner': 52.711873897820595}


 26%|██▌       | 7/27 [00:00<00:01, 11.57it/s]

In [ ]:


msg_val=pd.DataFrame()
for mail in os.listdir(email_dir_val):
  msg=extract_msg.Message(mail)
  msg_body=pd.Series(msg.body)
  msg_val=pd.concat([msg_val,msg_body])

In [ ]:
os.listdir(email_dir_val)

In [ ]:
msg_val[0]=msg_val[0].replace("\t"," ",regex=True).replace("\r\n"," ",regex=True).replace("\[EXTERNAL EMAIL]"," ",regex=True).replace("  "," ",regex=True)
print(msg_val.head())

In [ ]:
# test the trained model on validation dataset
for text in msg_val[0]:
    doc = ner_model(text)
    print(' Entities', [(ent.text, ent.label_) for ent in doc.ents])

In [91]:
import spacy
nlp=spacy.load('en')


In [94]:
for text in msg_val[0]:
  doc=nlp(text)
  print(' Entities', [(ent.text, ent.label_) for ent in doc.ents])

 Entities [('HUGHES MACHINERY CO 14400', 'ORG'), ('COLLEGE BLVD LENEXA', 'LAW'), ('66215', 'DATE'), ('4952743', 'CARDINAL'), ('1', 'CARDINAL'), ('44', 'CARDINAL'), ('44', 'CARDINAL'), ('38', 'CARDINAL'), ('70', 'CARDINAL'), ('048580', 'CARDINAL'), ('5', 'CARDINAL'), ('Tina Sternbergh', 'PERSON'), ('Armstrong International', 'PERSON'), ('269', 'CARDINAL'), ('279', 'CARDINAL'), ('Correspondence', 'PERSON'), ('RMA Request', 'PERSON')]
 Entities [('Afternoon', 'TIME'), ('BOL', 'ORG'), ('Corrosion Fluid', 'ORG'), ('1', 'CARDINAL'), ('323', 'CARDINAL'), ('60446', 'CARDINAL'), ('314', 'CARDINAL'), ('800‑678‑4858', 'CARDINAL'), ('Bridgeton', 'GPE'), ('MO', 'ORG'), ('TEXAS PIPE & SUPPLY', 'ORG'), ('MultAlloy', 'ORG'), ('Cuba', 'GPE'), ('Iran', 'GPE'), ('North Korea', 'GPE'), ('Sudan', 'GPE'), ('Syria', 'GPE')]
 Entities [('tomorrow', 'DATE'), ('25', 'CARDINAL'), ('1', 'CARDINAL'), ('490 pounds', 'QUANTITY'), ('4944661', 'CARDINAL'), ('85', 'CARDINAL'), ('Howard Gardner Denver-Fraser', 'PERSON')

In [16]:
doc=nlp(data[5]['text'])

In [58]:
data[5]['text']

'"Hello,   This is shipping from the TX warehouse Rockwell Medical 4051 Freeport Parkway  Grapevine, TX 76051 800-449-3353 ext 460   PO 46406238 OP (Ontario CA)  P/U# C582158 2 pallets 3,250lbs Ship date: 3/17/21"\r'

In [17]:
for ent in doc.ents:
  print(ent.text,ent.start_char,ent.end_char,ent.label_)

TX 36 38 ORG
Rockwell Medical 4051 Freeport Parkway 49 87 ORG
460 126 129 CARDINAL
P/U# C582158 2 161 175 ORG
